In [0]:
#spark.read.csv() 메소드를 이용하여 csv 파일을 로드하고 DataFrame으로 변환. 
# pandas_df = pd.read_csv('/FileStore/tables/titanic_train.csv', header='infer')
titanic_sdf = spark.read.csv('/FileStore/tables/titanic_train.csv', header=True, inferSchema=True)

# pandas DataFrame을 spark DataFrame으로 부터 생성. 
titanic_pdf = titanic_sdf.select('*').toPandas()

display(titanic_sdf.limit(10))

In [0]:
## SQL 사용이 가능할 수 있도록 titanic_sdf DataFrame을 DB View로 생성. 
titanic_sdf.createOrReplaceTempView('titanic_view')

In [0]:
%sql
select * from titanic_view 

In [0]:
titanic_sdf.select('*').show()

In [0]:
%sql

-- 10건만 추출
select * from titanic_view limit 10 

In [0]:
titanic_sdf.limit(10).show()
titanic_sdf.select('*').limit(10).show()

In [0]:
%sql

select upper(name) as cap_name, a.*  from titanic_view a


In [0]:
import pyspark.sql.functions as F

titanic_sdf.select(F.upper(F.col('name')).alias('cap_name'), '*').show()

In [0]:
%sql
-- cabin 컬럼의 첫번째 문자만 가져오는 cap_cabin 컬럼을 추가. substring() SQL 함수 활용
select a.*, substring(cabin, 0, 1) as cap_cabin from titanic_view a 


In [0]:
#titanic_sdf.select('*', F.substring(F.col('cabin'), 0, 1).alias('cap_cabin')).show()
titanic_sdf.withColumn('cap_cabin', F.substring(F.col('cabin'), 0, 1)).show()

In [0]:
%sql
select  * from titanic_view where Pclass = 1 and Age > 60 -- titanic_sdf.filter((F.col('pclass')==1) & (F.col('age') > 60)).show()

In [0]:
%sql
select pclass, name, age from titanic_view where upper(name) like '%MR.%' 

In [0]:
%sql
select * from titanic_view where pclass in (1, 2) 

In [0]:
%sql
-- order by 절로 정렬. pclass로 오름 차순 정렬 후 10건만 추출. 
select * from titanic_view order by pclass limit 10 -- titanic_sdf.orderBy('pclass').limit(10).show()

In [0]:
%sql
-- pclass는 오름차순, name은 내림차순으로 정렬
select * from titanic_view order by pclass asc, name desc

In [0]:
%sql
-- view의 건수 추출 
select count(*) as cnt from titanic_view -- titanic_sdf.count()

In [0]:
%sql
-- titanic_view의 최대 age, 최소 age, 평균 sibsp 값을 추출. 
select max(age), min(age), avg(sibsp) from titanic_view 


In [0]:
%sql
-- pclass로 group by 하여 pclass 레벨로 최대 age, 최소 age, 평균 sibsp 값을 추출. 
select pclass, max(age), min(age), avg(sibsp) from titanic_view group by pclass

In [0]:
%sql
-- pclass로 group by 하여 pclass 레벨로 최대 age, 최소 age, 평균 sibsp 값을 추출. 
select pclass, max(age) as max_age, min(age) as min_age, avg(sibsp) as avg_sibsp from titanic_view group by pclass having max(age) > 70
/* 
import pyspark.sql.functions as F
titanic_sdf.groupBy('pclass').agg(F.max(col('age')).alias('max_age'), 
                                  F.min(col('age')).alis('min_age'), 
                                  F.avg(col('sibsp')).alias('avg_sibsp')).filter(col('max_age') > 70).show()
*/

In [0]:
%sql

select pclass, max_age, min_age, avg_sibsp
from ( 
      select pclass, max(age) as max_age, min(age) as min_age, avg(sibsp) as avg_sibsp from titanic_view group by pclass
) where max_age > 70

In [0]:
%sql
-- 아래 update sql은 동작하지 않음. 
update titanic_view set fare=fare*10 -- titanic_sdf.withColumn('Fare', col('Fare')*10)

In [0]:
%sql

delete from titanic_view;

In [0]:
%sql
-- Null 조건(Not Null 조건 )
select * from titanic_view where age is null and cabin is not null -- titanic_sdf.filter(F.col('age').isNull & F.col('cabin').isNotNull())

In [0]:
import pyspark.sql.functions as F

titanic_sdf.filter((F.col('age').isNull()) & (F.col('cabin').isNotNull())).show()

In [0]:
%sql

-- select case when 조건으로 if elif else 조건에 따라 값 생성. 
Select a.*, CASE WHEN age <= 5 THEN 'Baby'
                 WHEN age <=12 Then 'Child'
                 WHEN age <= 18 THEN 'Teenage'
                 WHEN age <= 25 THEN 'Student'
                 WHEN age <=35 THEN 'Young Adult'
                 WHEN age <=60 THEN 'Adult'
                 WHEN age IS NULL THEN 'NA'
                 ELSE 'Elderly' END as Age_Category from titanic_view a

In [0]:
titanic_sdf.withColumn('Age_category', F.when(F.col('Age') <= 5, 'Baby')
                                      .when(F.col('Age') <= 12, 'Child')
                                      .when(F.col('Age') <= 18, 'Teenage')
                                      .when(F.col('Age') <= 25, 'Student')
                                      .when(F.col('Age') <= 35, 'Young Adult')
                                      .when(F.col('Age') <= 60, 'Adult')
                                      .when(F.col('Age').isNull(), 'NA')
                                      .otherwise('Elderly')).show()

### Databricks의 내장된 Graph 시각화 기능 사용하기. 
* 일반적으로 spark의 경우 spark DataFrame을 pandas Dataframe으로 변환 후 matplotlib 이나 seaborn을 적용하여 시각화
* Spark에서 Pandas API와 매우 유사하게 DataFrame을 사용할 수 있도록 해주는 Koalas 패키지가 있음. Spark 3.2 부터는 Pandas API로 Koalas 패키지가 통합됨. 
* Databricks는 Graph 시각화 기능이 내장되어 있음. 이를 이용하여 손쉽게 시각화 가능. 
* 시각화 기능을 적용할 때에는 클라이언트 레벨에서 너무 많은 데이터를 가공하지 않도록 유의(매우 많은 시간과 리소스 소모)

In [0]:
%sql
--bar chart
select pclass, count(*) from titanic_view group by pclass

In [0]:
## SQL뿐만 아니라 DataFrame도 display() 적용 시 Databricks 시각화 기능 적용 가능. 
display(titanic_sdf.groupBy('Pclass').count())

In [0]:
%sql
-- histogram
select survived from titanic_view

In [0]:
%sql
-- bar chart
select Survived, count(*) from titanic_view group by Survived order by 2 desc

In [0]:
%sql 
-- Sex 별로 생존/사망 건수를 비교하여 보여 주기. 
select Survived, Sex, count(*) from titanic_view group by survived, sex;

In [0]:
%sql
-- pclass와 sex 별로 생존/사망 건수를 비교하여 보여 주기 
select Survived, Sex, Pclass, count(*) cnt from titanic_view group by survived, sex, pclass;

In [0]:
%sql
-- scatter plot
select fare, age from titanic_view

In [0]:
%sql
-- boxplot
select age from titanic_view